### Алгоритм Weighted KNN
Для классифицируемого объекта x:
1. Вычислить расстояния до всех объектов обучающей выборки
2. Выбрать k ближайших соседей
3. Вычислить веса для каждого соседа
4. Для каждого класса c вычислить суммарный вес W_c = Σ w_i для всех соседей класса c
5. Назначить объекту x класс с наибольшим суммарным весом

### Преимущества перед стандартным KNN

Учет близости - ближайшие соседи влияют сильнее

Устойчивость к шуму - выбросы имеют меньший вес

Гибкость - можно использовать разные функции весов

Лучшая точность - особенно при неравномерной плотности данных

### Реализация Weighted KNN

In [ ]:
import numpy as np
from collections import Counter
from typing import Any, Union, List, Tuple, Optional
import math

class WeightedKNN:
    def __init__(self, k: int = 3, weight_type: str = 'inverse_distance') -> None:
        """
        Инициализация алгоритма взвешенного KNN
        
        Parameters:
        k (int): количество ближайших соседей
        weight_type (str): тип весовой функции
            - 'inverse_distance': обратное расстояние
            - 'inverse_square': обратный квадрат расстояния
            - 'gaussian': гауссово ядро
            - 'uniform': равномерные веса (стандартный KNN)
        """
        self.k = k
        self.weight_type = weight_type
        self.X_train = None
        self.y_train = None
        self.classes_ = None
    
    def euclidean_distance(self, point1: Union[np.ndarray, List[Union[float, int]]], 
                          point2: Union[np.ndarray, List[Union[float, int]]]) -> float:
        """
        Вычисление евклидова расстояния между двумя точками
        """
        return math.sqrt(sum((x - y) ** 2 for x, y in zip(point1, point2)))
    
    def _calculate_weight(self, distance: float) -> float:
        """
        Вычисление веса на основе расстояния
        
        Parameters:
        distance: расстояние до соседа
        
        Returns:
        float: вес соседа
        """
        epsilon = 1e-8  # чтобы избежать деления на 0
        
        if self.weight_type == 'uniform':
            return 1.0
        elif self.weight_type == 'inverse_distance':
            return 1.0 / (distance + epsilon)
        elif self.weight_type == 'inverse_square':
            return 1.0 / (distance ** 2 + epsilon)
        elif self.weight_type == 'gaussian':
            sigma = 1.0  # параметр ширины ядра
            return math.exp(-(distance ** 2) / (2 * sigma ** 2))
        else:
            raise ValueError(f"Неизвестный тип весов: {self.weight_type}")
    
    def fit(self, X: Union[np.ndarray, List[List[Union[float, int]]]], 
            y: Union[np.ndarray, List[Any]]) -> None:
        """
        Обучение модели - просто запоминаем данные
        
        Parameters:
        X: матрица признаков (n_samples, n_features)
        y: вектор меток (n_samples)
        """
        self.X_train = np.array(X)
        self.y_train = np.array(y)
        self.classes_ = np.unique(y)
    
    def predict_single(self, x: Union[np.ndarray, List[Union[float, int]]]) -> Any: 
        """
        Предсказание для одного образца с использованием взвешенного голосования
        
        Parameters:
        x: вектор признаков одного образца
        
        Returns:
        predicted_label: предсказанная метка
        """
        # Вычисляем расстояния до всех тренировочных образцов
        distances = []
        for i, train_point in enumerate(self.X_train):
            dist = self.euclidean_distance(x, train_point)
            distances.append((dist, self.y_train[i], i))
        
        # Сортируем по расстоянию и берем k ближайших
        distances.sort(key=lambda x: x[0])
        k_nearest = distances[:self.k]
        
        # Взвешенное голосование
        class_weights = {cls: 0.0 for cls in self.classes_}
        
        for dist, label, idx in k_nearest:
            weight = self._calculate_weight(dist)
            class_weights[label] += weight
        
        # Выбираем класс с наибольшим суммарным весом
        predicted_class = max(class_weights.items(), key=lambda x: x[1])[0]
        return predicted_class
    
    def predict_proba_single(self, x: Union[np.ndarray, List[Union[float, int]]]) -> np.array:
        """
        Предсказание вероятностей для одного образца
        
        Parameters:
        x: вектор признаков одного образца
        
        Returns:
        probabilities: вероятности для каждого класса
        """
        # Вычисляем расстояния до всех тренировочных образцов
        distances = []
        for i, train_point in enumerate(self.X_train):
            dist = self.euclidean_distance(x, train_point)
            distances.append((dist, self.y_train[i], i))
        
        # Сортируем по расстоянию и берем k ближайших
        distances.sort(key=lambda x: x[0])
        k_nearest = distances[:self.k]
        
        # Вычисляем веса для каждого класса
        class_weights = {cls: 0.0 for cls in self.classes_}
        total_weight = 0.0
        
        for dist, label, idx in k_nearest:
            weight = self._calculate_weight(dist)
            class_weights[label] += weight
            total_weight += weight
        
        # Нормализуем веса для получения вероятностей
        probabilities = np.array([class_weights[cls] / total_weight for cls in self.classes_])
        return probabilities
    
    def predict(self, X: Union[np.ndarray, List[List[Union[float, int]]]]) -> np.array:
        """
        Предсказание для набора образцов
        
        Parameters:
        X: матрица признаков для предсказания
        
        Returns:
        predictions: массив предсказанных меток
        """
        predictions = []
        for x in X:
            predictions.append(self.predict_single(x))
        return np.array(predictions)
    
    def predict_proba(self, X: Union[np.ndarray, List[List[Union[float, int]]]]) -> np.array:
        """
        Предсказание вероятностей для набора образцов
        
        Parameters:
        X: матрица признаков для предсказания
        
        Returns:
        probabilities: матрица вероятностей (n_samples, n_classes)
        """
        probabilities = []
        for x in X:
            probabilities.append(self.predict_proba_single(x))
        return np.array(probabilities)
    
    def accuracy(self, X_test: Union[np.ndarray, List[List[Union[float, int]]]], 
                 y_test: Union[np.ndarray, List[Any]]) -> float:
        """
        Вычисление точности модели
        
        Parameters:
        X_test: тестовые данные
        y_test: истинные метки тестовых данных
        
        Returns:
        float: точность модели
        """
        predictions = self.predict(X_test)
        correct = sum(predictions == y_test)
        return correct / len(y_test)
    
    def get_neighbors_info(self, x: Union[np.ndarray, List[Union[float, int]]]) -> List[Tuple]:
        """
        Получение информации о k ближайших соседях
        
        Parameters:
        x: вектор признаков одного образца
        
        Returns:
        list: информация о соседях (расстояние, метка, вес, индекс)
        """
        distances = []
        for i, train_point in enumerate(self.X_train):
            dist = self.euclidean_distance(x, train_point)
            distances.append((dist, self.y_train[i], i))
        
        distances.sort(key=lambda x: x[0])
        k_nearest = distances[:self.k]
        
        neighbors_info = []
        for dist, label, idx in k_nearest:
            weight = self._calculate_weight(dist)
            neighbors_info.append({
                'distance': dist,
                'label': label,
                'weight': weight,
                'index': idx
            })
        
        return neighbors_info



### Пример использования реализованного Weighted KNN

In [21]:
# Демонстрация работы взвешенного KNN
def demo_weighted_knn():
    """
    Демонстрация различных типов весов в KNN
    """
    # Создаем простой dataset
    from sklearn.datasets import make_classification
    from sklearn.model_selection import train_test_split
    
    # Генерируем данные
    X, y = make_classification(
        n_samples=100,
        n_features=2,
        n_redundant=0,
        n_informative=2,
        n_clusters_per_class=1,
        random_state=42
    )
    
    # Разделяем на train/test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Сравниваем разные типы весов
    weight_types = ['uniform', 'inverse_distance', 'inverse_square', 'gaussian']
    
    print("Сравнение различных типов весов в KNN:")
    print("=" * 50)
    
    for weight_type in weight_types:
        knn = WeightedKNN(k=5, weight_type=weight_type)
        knn.fit(X_train, y_train)
        accuracy = knn.accuracy(X_test, y_test)
        print(f"{weight_type:>15}: Точность = {accuracy:.4f}")
    
    # Демонстрация предсказания вероятностей
    print("\nДемонстрация predict_proba:")
    print("=" * 50)
    
    knn = WeightedKNN(k=3, weight_type='inverse_distance')
    knn.fit(X_train, y_train)
    
    # Предсказание для первого тестового образца
    sample = X_test[0]
    true_label = y_test[0]
    
    prediction = knn.predict_single(sample)
    probabilities = knn.predict_proba_single(sample)
    neighbors_info = knn.get_neighbors_info(sample)
    
    print(f"Образец: {sample}")
    print(f"Истинная метка: {true_label}")
    print(f"Предсказанная метка: {prediction}")
    print(f"Вероятности: {probabilities}")
    print(f"Вероятность класса 0: {probabilities[0]:.4f}")
    print(f"Вероятность класса 1: {probabilities[1]:.4f}")
    
    print("\nБлижайшие соседи:")
    for i, neighbor in enumerate(neighbors_info):
        print(f"  Сосед {i+1}: расстояние={neighbor['distance']:.4f}, "
              f"метка={neighbor['label']}, вес={neighbor['weight']:.4f}")


if __name__ == "__main__":
    demo_weighted_knn()

Сравнение различных типов весов в KNN:
        uniform: Точность = 1.0000
inverse_distance: Точность = 1.0000
 inverse_square: Точность = 1.0000
       gaussian: Точность = 1.0000

Демонстрация predict_proba:
Образец: [ 1.68674524 -0.35904111]
Истинная метка: 0
Предсказанная метка: 0
Вероятности: [1. 0.]
Вероятность класса 0: 1.0000
Вероятность класса 1: 0.0000

Ближайшие соседи:
  Сосед 1: расстояние=0.1526, метка=0, вес=6.5525
  Сосед 2: расстояние=0.1529, метка=0, вес=6.5408
  Сосед 3: расстояние=0.1805, метка=0, вес=5.5400


## Пример использования встроенного Weighted KNN в sklearn

### Сравнение результатов работы Uniform KNN (стандартный) и Weighted KNN (взвешенный)

In [26]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

# Загружаем данные
iris = load_iris()
X, y = iris.data, iris.target
feature_names = iris.feature_names
target_names = iris.target_names

print("Информация о датасете Iris:")
print(f"Количество образцов: {X.shape[0]}")
print(f"Количество признаков: {X.shape[1]}")
print(f"Признаки: {feature_names}")
print(f"Классы: {target_names}")
print()

# Разделяем данные на тренировочные и тестовые
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Масштабируем данные
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Размеры данных:")
print(f"Тренировочные данные: {X_train_scaled.shape}")
print(f"Тестовые данные: {X_test_scaled.shape}")
print()

# Создаем модели с разными типами весов
models = {
    'Uniform KNN (стандартный)': KNeighborsClassifier(n_neighbors=5, weights='uniform'),
    'Weighted KNN (взвешенный)': KNeighborsClassifier(n_neighbors=5, weights='distance')
}

# Обучаем и оцениваем модели
results = {}

for name, model in models.items():
    print(f"Обучение модели: {name}")
    
    # Обучение
    model.fit(X_train_scaled, y_train)
    
    # Предсказание
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)
    
    # Оценка точности
    accuracy = accuracy_score(y_test, y_pred)
    
    results[name] = {
        'model': model,
        'predictions': y_pred,
        'probabilities': y_pred_proba,
        'accuracy': accuracy
    }
    
    print(f"Точность: {accuracy:.4f}")
    print()

# Сравниваем результаты
print("СРАВНЕНИЕ РЕЗУЛЬТАТОВ:")
print("=" * 50)
for name, result in results.items():
    print(f"{name}: {result['accuracy']:.4f}")

# Выводим предсказанные и фактические метки для всех тестовых образцов
print(f"{'№':<3} {'Модель':<25} {'Предсказание':<15} {'Фактическое':<15} {'Результат':<10}")
print("-" * 70)

for i in range(len(X_test)):
    for name, result in results.items():
        pred_label = result['predictions'][i]
        true_label = y_test[i]
        is_correct = pred_label == true_label
        status = "✓ ВЕРНО" if is_correct else "✗ ОШИБКА"
        
        print(f"{i+1:<3} {name:<25} {target_names[pred_label]:<15} {target_names[true_label]:<15} {status:<10}")
    print("-" * 70)


Информация о датасете Iris:
Количество образцов: 150
Количество признаков: 4
Признаки: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Классы: ['setosa' 'versicolor' 'virginica']

Размеры данных:
Тренировочные данные: (105, 4)
Тестовые данные: (45, 4)

Обучение модели: Uniform KNN (стандартный)
Точность: 0.9111

Обучение модели: Weighted KNN (взвешенный)
Точность: 0.9333

СРАВНЕНИЕ РЕЗУЛЬТАТОВ:
Uniform KNN (стандартный): 0.9111
Weighted KNN (взвешенный): 0.9333
№   Модель                    Предсказание    Фактическое     Результат 
----------------------------------------------------------------------
1   Uniform KNN (стандартный) virginica       virginica       ✓ ВЕРНО   
1   Weighted KNN (взвешенный) virginica       virginica       ✓ ВЕРНО   
----------------------------------------------------------------------
2   Uniform KNN (стандартный) versicolor      versicolor      ✓ ВЕРНО   
2   Weighted KNN (взвешенный) versicolor      versicolor      ✓ В